In [5]:
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image, ImageDraw
import PIL
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

try:
    os.mkdir('data')
except:
    print('Path Already Exists')

width, height = 500, 500

# Create main window
win = tk.Tk()
win.title("Handwritten Digit Recognition")
win.configure(bg='#1e1e2e')
win.resizable(False, False)

# Modern color scheme
bg_color = '#1e1e2e'
canvas_bg = '#ffffff'
primary_color = '#89b4fa'
success_color = '#a6e3a1'
warning_color = '#f9e2af'
danger_color = '#f38ba8'
text_color = '#cdd6f4'

# Modern fonts
font_btn = ('Segoe UI', 11, 'bold')
font_label = ('Segoe UI', 14, 'bold')
font_title = ('Segoe UI', 18, 'bold')

count = 0

import joblib

clsfr = joblib.load('number-character-knn.sav')

label_dict = {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9'}

def event_function(event):
    x = event.x
    y = event.y
    
    x1 = x - 20
    y1 = y - 20
    x2 = x + 20
    y2 = y + 20
    
    canvas.create_oval((x1, y1, x2, y2), fill='black', outline='black')
    img_draw.ellipse((x1, y1, x2, y2), fill='black')

def save():
    global count
    
    img_array = np.array(img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    img_array = cv2.bitwise_not(img_array)
    img_array = cv2.resize(img_array, (28, 28))

    path = os.path.join('data', str(count) + '.jpg')
    cv2.imwrite(path, img_array)

    count = count + 1
    label_status.config(text=f'✓ Saved as image #{count}')

def clear():
    global img, img_draw

    canvas.delete('all')
    img = Image.new('RGB', (width, height), (255, 255, 255))
    img_draw = ImageDraw.Draw(img)
    label_status.config(text='Ready to draw...')

def predict():
    img_array = np.array(img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    img_array = cv2.bitwise_not(img_array)
    img_array = cv2.resize(img_array, (28, 28))
    
    img_array = np.reshape(img_array, (1, 784))

    result = clsfr.predict(img_array)[0]
    label = label_dict[result]
    label_status.config(text=f'Predicted Digit: {label}')
    
    plt.show()

# Title frame
title_frame = tk.Frame(win, bg=bg_color, pady=15)
title_frame.grid(row=0, column=0, columnspan=4, sticky='ew')

title_label = tk.Label(
    title_frame, 
    text='✍ Digit Recognition', 
    bg=bg_color, 
    fg=primary_color,
    font=font_title
)
title_label.pack()

subtitle_label = tk.Label(
    title_frame,
    text='Draw a digit (0-9) in the canvas below',
    bg=bg_color,
    fg=text_color,
    font=('Segoe UI', 10)
)
subtitle_label.pack()

# Canvas with modern border
canvas_frame = tk.Frame(win, bg=bg_color, padx=20, pady=10)
canvas_frame.grid(row=1, column=0, columnspan=4)

canvas = tk.Canvas(
    canvas_frame,
    width=width,
    height=height,
    bg=canvas_bg,
    highlightthickness=2,
    highlightbackground=primary_color,
    cursor='pencil'
)
canvas.pack()

# Button frame
button_frame = tk.Frame(win, bg=bg_color, pady=15)
button_frame.grid(row=2, column=0, columnspan=4)

# Modern styled buttons
button_save = tk.Button(
    button_frame,
    text='💾 SAVE',
    bg=success_color,
    fg='#1e1e2e',
    font=font_btn,
    command=save,
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor='hand2',
    activebackground='#94e2ad',
    activeforeground='#1e1e2e'
)
button_save.grid(row=0, column=0, padx=8)

button_predict = tk.Button(
    button_frame,
    text='🔮 PREDICT',
    bg=primary_color,
    fg='#1e1e2e',
    font=font_btn,
    command=predict,
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor='hand2',
    activebackground='#a0c4fa',
    activeforeground='#1e1e2e'
)
button_predict.grid(row=0, column=1, padx=8)

button_clear = tk.Button(
    button_frame,
    text='🗑️ CLEAR',
    bg=warning_color,
    fg='#1e1e2e',
    font=font_btn,
    command=clear,
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor='hand2',
    activebackground='#fae8b8',
    activeforeground='#1e1e2e'
)
button_clear.grid(row=0, column=2, padx=8)

button_exit = tk.Button(
    button_frame,
    text='❌ EXIT',
    bg=danger_color,
    fg='#1e1e2e',
    font=font_btn,
    command=win.destroy,
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor='hand2',
    activebackground='#f5a0b0',
    activeforeground='#1e1e2e'
)
button_exit.grid(row=0, column=3, padx=8)

# Status frame
status_frame = tk.Frame(win, bg=bg_color, pady=15)
status_frame.grid(row=3, column=0, columnspan=4, sticky='ew')

label_status = tk.Label(
    status_frame,
    text='Ready to draw...',
    bg=bg_color,
    fg=text_color,
    font=font_label,
    pady=10
)
label_status.pack()

# Bind canvas events
canvas.bind('<B1-Motion>', event_function)

# Initialize drawing
img = Image.new('RGB', (width, height), (255, 255, 255))
img_draw = ImageDraw.Draw(img)

# Center window on screen
win.update_idletasks()
window_width = win.winfo_width()
window_height = win.winfo_height()
screen_width = win.winfo_screenwidth()
screen_height = win.winfo_screenheight()
x = (screen_width // 2) - (window_width // 2)
y = (screen_height // 2) - (window_height // 2)
win.geometry(f'+{x}+{y}')

win.mainloop()


Path Already Exists
